# DESI E-Value Analysis: Testing Dark Energy Evidence

This notebook analyzes the DESI DR2 BAO measurements using e-values to compare
LCDM against dynamic dark energy (w0waCDM) models.

## Background

The DESI collaboration reported 2.8-4.2 sigma evidence for dynamic dark energy
in their DR2 results (March 2025). However, this evidence has been criticized:

1. **Bayesian analysis favors LCDM** - ln B = -0.57 for DESI+CMB (arxiv:2511.10631)
2. **Dataset tensions** - CMB, BAO, SNe show internal inconsistencies
3. **w0waCDM may just resolve tensions** rather than detect new physics

We use e-values to provide another perspective on this evidence.

## References
- DESI DR2: [arXiv:2503.14738](https://arxiv.org/abs/2503.14738)
- E-values: Ramdas et al. (2023), Statistical Science 38(4)
- Bayesian critique: [arXiv:2511.10631](https://arxiv.org/abs/2511.10631)

In [ ]:
import sys
sys.path.insert(0, '../code')

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from cosmology import (
    CosmologyParams, LCDM, DESI_DR2_BEST_FIT,
    compute_bao_predictions, chi_squared, log_likelihood,
    DM, DH, DV
)
from data_loader import load_desi_data, get_theory_vector, BAODataset
from evalue_analysis import (
    likelihood_ratio_evalue, split_evalue, grow_evalue,
    summarize_evidence_caveats, _build_theory_vector
)

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

DATA_DIR = Path('../data')

## 1. Load DESI Data

We load both DR1 and DR2 BAO measurements from the official Cobaya data files.

In [ ]:
# Load datasets
dr1 = load_desi_data(DATA_DIR / 'dr1', 'DR1')
dr2 = load_desi_data(DATA_DIR / 'dr2', 'DR2')

print("DESI DR1 Data:")
print(dr1.summary())
print("\n" + "="*60 + "\n")
print("DESI DR2 Data:")
print(dr2.summary())

## 2. Visualize BAO Measurements

Compare DESI measurements against LCDM and w0waCDM predictions.

In [ ]:
# Generate theory predictions
z_theory = np.linspace(0.1, 2.5, 100)

pred_lcdm = compute_bao_predictions(z_theory, LCDM)
pred_w0wa = compute_bao_predictions(z_theory, DESI_DR2_BEST_FIT)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# DM/rd plot
ax = axes[0]
ax.plot(z_theory, pred_lcdm['DM_over_rd'], 'b-', lw=2, label='LCDM')
ax.plot(z_theory, pred_w0wa['DM_over_rd'], 'r--', lw=2, label='w0waCDM (DESI best-fit)')

# DR2 data
dm_mask = ['DM' in q for q in dr2.quantities]
ax.errorbar(
    dr2.z_eff[dm_mask], 
    dr2.data[dm_mask], 
    yerr=dr2.errors[dm_mask],
    fmt='ko', capsize=3, markersize=8, label='DESI DR2'
)

ax.set_xlabel('Redshift z')
ax.set_ylabel(r'$D_M / r_d$')
ax.set_title('Transverse Comoving Distance')
ax.legend()

# DH/rd plot
ax = axes[1]
ax.plot(z_theory, pred_lcdm['DH_over_rd'], 'b-', lw=2, label='LCDM')
ax.plot(z_theory, pred_w0wa['DH_over_rd'], 'r--', lw=2, label='w0waCDM')

dh_mask = ['DH' in q for q in dr2.quantities]
ax.errorbar(
    dr2.z_eff[dh_mask], 
    dr2.data[dh_mask], 
    yerr=dr2.errors[dh_mask],
    fmt='ko', capsize=3, markersize=8, label='DESI DR2'
)

ax.set_xlabel('Redshift z')
ax.set_ylabel(r'$D_H / r_d$')
ax.set_title('Hubble Distance')
ax.legend()

plt.tight_layout()
plt.savefig('../docs/bao_measurements.png', dpi=150)
plt.show()

## 3. Chi-Squared Analysis

First, let's do a traditional chi-squared comparison between models.

In [ ]:
# Compute theory predictions at data redshifts
pred_lcdm_data = compute_bao_predictions(dr2.z_eff, LCDM)
pred_w0wa_data = compute_bao_predictions(dr2.z_eff, DESI_DR2_BEST_FIT)

theory_lcdm = _build_theory_vector(pred_lcdm_data, dr2.z_eff, dr2.quantities)
theory_w0wa = _build_theory_vector(pred_w0wa_data, dr2.z_eff, dr2.quantities)

chi2_lcdm = chi_squared(dr2.data, theory_lcdm, dr2.cov)
chi2_w0wa = chi_squared(dr2.data, theory_w0wa, dr2.cov)

n_data = len(dr2.data)
dof_lcdm = n_data - 0  # LCDM has no free parameters in this comparison
dof_w0wa = n_data - 2  # w0waCDM has 2 extra parameters

print("Chi-Squared Analysis (DESI DR2)")
print("="*50)
print(f"N data points: {n_data}")
print(f"\nLCDM:")
print(f"  chi2 = {chi2_lcdm:.2f}")
print(f"  chi2/dof = {chi2_lcdm/dof_lcdm:.2f}")
print(f"\nw0waCDM (DESI best-fit w0={DESI_DR2_BEST_FIT.w0}, wa={DESI_DR2_BEST_FIT.wa}):")
print(f"  chi2 = {chi2_w0wa:.2f}")
print(f"  chi2/dof = {chi2_w0wa/dof_w0wa:.2f}")
print(f"\nDelta chi2 = {chi2_lcdm - chi2_w0wa:.2f}")
print(f"Equivalent sigma (2 dof): {np.sqrt(chi2_lcdm - chi2_w0wa):.1f} sigma")

## 4. E-Value Analysis

Now compute e-values using different methods.

### 4.1 Simple Likelihood Ratio E-Value

The simplest e-value is just the likelihood ratio. This can overfit to data.

In [ ]:
e_simple = likelihood_ratio_evalue(
    dr2.data, dr2.cov, theory_lcdm, theory_w0wa
)

print("Simple Likelihood Ratio E-Value")
print("="*50)
print(f"E = {e_simple.e_value:.2f}")
print(f"log(E) = {e_simple.log_e:.2f}")
print(f"Sigma equivalent: {e_simple.sigma_equivalent:.1f}")
print(f"\nWARNING: This e-value uses the DESI best-fit as alternative,")
print(f"which was fitted to THIS data. This inflates the e-value!")

### 4.2 GROW Mixture E-Value

A more principled approach using a mixture over plausible alternatives.

In [ ]:
e_grow = grow_evalue(
    dr2.data, dr2.cov, dr2.z_eff, dr2.quantities
)

print("GROW Mixture E-Value")
print("="*50)
print(f"E = {e_grow.e_value:.2f}")
print(f"log(E) = {e_grow.log_e:.2f}")
print(f"Sigma equivalent: {e_grow.sigma_equivalent:.1f}")
print(f"\nBest-fit alternative:")
print(f"  w0 = {e_grow.alt_params.w0:.3f}")
print(f"  wa = {e_grow.alt_params.wa:.3f}")

### 4.3 Data-Split E-Value

Split data into training (fit alternative) and test (evaluate e-value) sets.

In [ ]:
try:
    e_train, e_test = split_evalue(
        dr2.data, dr2.cov, dr2.z_eff, dr2.quantities,
        split_z=1.0  # Use z<1 to train, z>=1 to test
    )
    
    print("Data-Split E-Value Analysis")
    print("="*50)
    print(f"Split at z = 1.0")
    print(f"\nTraining set (z < 1.0):")
    print(f"  E = {e_train.e_value:.2f} (NOT for inference - overfitted)")
    print(f"  Fitted: w0={e_train.alt_params.w0:.3f}, wa={e_train.alt_params.wa:.3f}")
    print(f"\nTest set (z >= 1.0) - VALID FOR INFERENCE:")
    print(f"  E = {e_test.e_value:.2f}")
    print(f"  log(E) = {e_test.log_e:.2f}")
    print(f"  Sigma equivalent: {e_test.sigma_equivalent:.1f}")
except ValueError as e:
    print(f"Split analysis failed: {e}")
    print("Need more data points in each split.")

## 5. DR1 vs DR2 Comparison

Compare evidence from DR1 to DR2 to see how it evolved.

In [ ]:
# DR1 analysis
e_grow_dr1 = grow_evalue(
    dr1.data, dr1.cov, dr1.z_eff, dr1.quantities
)

print("Comparison: DR1 vs DR2")
print("="*50)
print(f"\nDR1 ({len(dr1.data)} measurements):")
print(f"  E = {e_grow_dr1.e_value:.2f}")
print(f"  Sigma equivalent: {e_grow_dr1.sigma_equivalent:.1f}")
print(f"  Best-fit: w0={e_grow_dr1.alt_params.w0:.3f}, wa={e_grow_dr1.alt_params.wa:.3f}")

print(f"\nDR2 ({len(dr2.data)} measurements):")
print(f"  E = {e_grow.e_value:.2f}")
print(f"  Sigma equivalent: {e_grow.sigma_equivalent:.1f}")
print(f"  Best-fit: w0={e_grow.alt_params.w0:.3f}, wa={e_grow.alt_params.wa:.3f}")

print(f"\nRatio E_DR2 / E_DR1 = {e_grow.e_value / e_grow_dr1.e_value:.2f}")

## 6. Critical Assessment

Now let's discuss why e-values may be misleading here.

In [ ]:
print(summarize_evidence_caveats(e_grow))

## 7. Sensitivity Analysis

How sensitive are the e-values to the choice of alternative hypothesis range?

In [ ]:
# Test sensitivity to prior range
w0_ranges = [
    (-1.2, -0.8),  # Narrow around LCDM
    (-1.5, -0.5),  # Moderate (our default)
    (-2.0, 0.0),   # Wide
]

wa_ranges = [
    (-1.0, 0.5),   # Narrow
    (-2.0, 1.0),   # Moderate (our default)
    (-3.0, 2.0),   # Wide
]

print("Sensitivity to Prior Range")
print("="*60)
print(f"{'w0 range':>15} {'wa range':>15} {'E-value':>12} {'log(E)':>10}")
print("-"*60)

for w0_r, wa_r in zip(w0_ranges, wa_ranges):
    # Recompute with different grid
    w0_grid = np.linspace(w0_r[0], w0_r[1], 10)
    wa_grid = np.linspace(wa_r[0], wa_r[1], 10)
    
    # Compute likelihood ratios
    log_ratios = []
    pred_null = compute_bao_predictions(dr2.z_eff, LCDM)
    theory_null = _build_theory_vector(pred_null, dr2.z_eff, dr2.quantities)
    log_L_null = log_likelihood(dr2.data, theory_null, dr2.cov)
    
    for w0 in w0_grid:
        for wa in wa_grid:
            cosmo = CosmologyParams(w0=w0, wa=wa)
            pred = compute_bao_predictions(dr2.z_eff, cosmo)
            theory_alt = _build_theory_vector(pred, dr2.z_eff, dr2.quantities)
            log_L_alt = log_likelihood(dr2.data, theory_alt, dr2.cov)
            log_ratios.append(log_L_alt - log_L_null)
    
    log_ratios = np.array(log_ratios)
    max_lr = np.max(log_ratios)
    log_e = max_lr + np.log(np.mean(np.exp(log_ratios - max_lr)))
    e_val = np.exp(log_e)
    
    print(f"{str(w0_r):>15} {str(wa_r):>15} {e_val:>12.2f} {log_e:>10.2f}")

print("\nNote: E-value varies significantly with prior choice!")
print("This is a fundamental limitation of the approach.")

## 8. Visualization of Evidence Trajectory

How does the e-value accumulate as we add data points?

In [ ]:
# Sequential evidence accumulation
# Sort by redshift and accumulate
sort_idx = np.argsort(dr2.z_eff)

cumulative_log_e = []
z_cumulative = []

for n in range(2, len(dr2.data) + 1):
    idx = sort_idx[:n]
    data_sub = dr2.data[idx]
    cov_sub = dr2.cov[np.ix_(idx, idx)]
    z_sub = dr2.z_eff[idx]
    q_sub = [dr2.quantities[i] for i in idx]
    
    # Compute GROW e-value on subset
    w0_grid = np.linspace(-1.5, -0.5, 8)
    wa_grid = np.linspace(-2.0, 1.0, 8)
    
    log_ratios = []
    pred_null = compute_bao_predictions(z_sub, LCDM)
    theory_null = _build_theory_vector(pred_null, z_sub, q_sub)
    log_L_null = log_likelihood(data_sub, theory_null, cov_sub)
    
    for w0 in w0_grid:
        for wa in wa_grid:
            cosmo = CosmologyParams(w0=w0, wa=wa)
            pred = compute_bao_predictions(z_sub, cosmo)
            theory_alt = _build_theory_vector(pred, z_sub, q_sub)
            log_L_alt = log_likelihood(data_sub, theory_alt, cov_sub)
            log_ratios.append(log_L_alt - log_L_null)
    
    log_ratios = np.array(log_ratios)
    max_lr = np.max(log_ratios)
    log_e = max_lr + np.log(np.mean(np.exp(log_ratios - max_lr)))
    
    cumulative_log_e.append(log_e)
    z_cumulative.append(z_sub[-1])

fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(z_cumulative, cumulative_log_e, 'b-o', lw=2, markersize=6)
ax.axhline(0, color='k', linestyle='--', alpha=0.5, label='No evidence (E=1)')
ax.axhline(np.log(10), color='orange', linestyle=':', alpha=0.7, label='E=10')
ax.axhline(np.log(100), color='red', linestyle=':', alpha=0.7, label='E=100')

ax.set_xlabel('Maximum redshift included')
ax.set_ylabel('log(E-value)')
ax.set_title('Cumulative Evidence vs Redshift')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../docs/evidence_trajectory.png', dpi=150)
plt.show()

## 9. Conclusions

### Summary of Results

In [ ]:
print("="*70)
print("DESI E-VALUE ANALYSIS: SUMMARY")
print("="*70)

print(f"\n1. DESI DR2 BAO Measurements:")
print(f"   - {len(dr2.data)} data points from z=0.295 to z=2.33")
print(f"   - Tracers: BGS, LRG, ELG, QSO, Lyman-alpha")

print(f"\n2. Chi-Squared Results:")
print(f"   - LCDM chi2 = {chi2_lcdm:.1f}")
print(f"   - w0waCDM chi2 = {chi2_w0wa:.1f}")
print(f"   - Delta chi2 = {chi2_lcdm - chi2_w0wa:.1f}")

print(f"\n3. E-Value Results:")
print(f"   - Simple LR: E = {e_simple.e_value:.1f} (BIASED - overfitted)")
print(f"   - GROW mixture: E = {e_grow.e_value:.1f} (~{e_grow.sigma_equivalent:.1f} sigma)")

print(f"\n4. KEY CAVEATS:")
print(f"   a) Bayesian analysis FAVORS LCDM (ln B = -0.57 for DESI+CMB)")
print(f"   b) E-values sensitive to prior range (varied 2-3x in our tests)")
print(f"   c) Dataset tensions may drive apparent signal")
print(f"   d) w0waCDM may be resolving tensions, not detecting physics")

print(f"\n5. RECOMMENDATION:")
print(f"   The evidence for dynamic dark energy is NOT robust.")
print(f"   The disagreement between frequentist (3-4 sigma) and Bayesian")
print(f"   (favors LCDM) analyses suggests systematic issues.")
print(f"   ")
print(f"   Wait for:")
print(f"   - DESI DR3+ with more data")
print(f"   - Resolution of dataset tensions")
print(f"   - Independent confirmation (Euclid, Roman)")
print("="*70)